##Эвотор Data Challenge. Задача классификации товаров по категориям

##### Автор: Мартынов Роман

##### Данное довольно простое решение дало наилучший результат, которого я смог добиться. Он соответствует 20 месту в финальном рейтинге и равен 0,947752 (accuracy)

In [7]:
import re
import pickle
import pymorphy2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ShuffleSplit
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

##### Парсинг исходных данных

Поскольку в файлах .csv некоторые строки были разделены на несколько строк, было принято решение вручную распарсить эти файлы

In [8]:
def categories_parse(path: 'str') -> (list, list):
    """Парсер файла categories.csv"""
    file = open(path, 'r', encoding='utf-8')
    file.readline()
    s = ''
    group_id = []
    name = []

    for line in file:
        # если одна строка csv разнесена на две, то конкатенируем с предыдущей считавшейся
        s += line
        p = re.compile(r'([\d]+)'                    # число
                       r'[,]'                        # запятая
                       r'["]'                        # открывающаяся кавычка
                       r'([^"]+)'                    # любые символы, кроме "
                       r'["]'                        # закрывающаяся кавычка
                       r'[\w | \W]+')                # любые символы
        m = p.search(s)
        if m is None:
            pass
        else:
            s = ''
            group_id.append(m.group(1))
            name.append(m.group(2))
    return group_id, name

def train_parse(path: '') -> (list, list, list):
    """Парсер файла evo_train.csv"""
    file = open(path, 'r', encoding='utf-8')
    file.readline()
    s = ''
    name = []
    group_id = []
    id = []
    
    for line in file:
        s += line
        p = re.compile(r'([\w | \W]+)'    # любой символ любое число раз
                       r'[,]'             # запятая
                       r'([\d]+)'         # число
                       r'[,]'             # запятая
                       r'([\d]+)'         # число
                       r'$')              # конец строки
        m = p.search(s)
        if m is None:
            pass
        else:
            s = ''
            name.append(m.group(1))
            group_id.append(m.group(2))
            id.append(m.group(3))
    return name, group_id, id

def test_parse(path: '') -> (list, list, list):
    """Парсер файла evo_test.csv"""
    file = open(path, 'r', encoding='utf-8')
    file.readline()
    s = ''
    name = []
    id = []

    for line in file:
        s += line
        p = re.compile(r'([\w | \W]+)'    # любой символ любое число раз
                       r'[,]'             # запятая
                       r'([\d]+)'         # число
                       r'$')              # конец строки
        m = p.search(s)
        if m is None:
            pass
        else:
            s = ''
            name.append(m.group(1))
            id.append(m.group(2))
    return name, id

# парсинг тренировочных данных
Data_train, group_id_tr, id_tr = train_parse(r'data\evo_train.csv')
# парсинг тестовых данных
Data_test, id_test = test_parse(r'data\evo_test.csv')

##### Предобработка исходных данных

В случаях применения стемминга или лемматизации, качество предсказаний резко падало, поэтому от этих способов пришлось отказаться.

In [9]:
class Text_Preprocessor():

    def __init__(self, text):
        # текст = [[word, ... , word], [word, ... ,word]]
        self.format_text = [line.lower().split(' ') for line in text]

    def del_word_with_n_letters(self, n=1):
        """Удаление слов, состоящих из одной буквы"""
        res_text = []
        for line in self.format_text:
            line_list = []
            for word in line:
                if len(word) > n:
                    line_list.append(word)
            res_text.append(line_list)
        self.format_text = res_text

    def only_words(self):
        """Оставляет только слова кириллицей или латиницей"""
        res_text = []
        p = re.compile(r'[а-яА-ЯёЁa-zA-Z]+')
        for line in self.format_text:
            line_list = []
            for word in line:
                if p.findall(word):
                    for elem in p.findall(word):
                        line_list.append(elem)
            res_text.append(line_list)
        self.format_text = res_text

    def del_stop_words(self):
        """Удаляет предлоги"""
        morph = pymorphy2.MorphAnalyzer()
        res_text = []
        for line in self.format_text:
            new_line = []
            for word in line:
                p = morph.parse(word)[0]
                pos = p.tag.POS
                if pos not in ['PREP']:
                    new_line.append(word)
            res_text.append(new_line)
        self.format_text = res_text

    def zamena(self):
        """Замены символа й на и, ё на е"""
        p = re.compile('й')
        r = re.compile('ё')
        res_text = []
        for line in self.format_text:
            new_line = []
            for word in line:
                word = p.sub('и', word)
                word = r.sub('е', word)
                new_line.append(word)
            res_text.append(new_line)
        self.format_text = res_text

    def processing(self):
        self.only_words()
        self.del_word_with_n_letters(n=1)
        #self.del_stop_words()
        self.zamena()
        # текст = ['строка', 'строка', ... , 'строка']
        self.list_text = [' '.join(spisok) for spisok in self.format_text]

# предобработка тренировочных данных
X_train = Text_Preprocessor(Data_train)
X_train.processing()
# предобработка тестовых данных
X_test = Text_Preprocessor(Data_test)
X_test.processing()

##### Машинное обучение

In [10]:
class MLearning():

    def __init__(self, X_data, y_target):
        self.X_data = X_data
        self.y_target = y_target

    def train_clf(self):
        # объединяем векторизатор TF-IDF и классификатор SVM в единый конвейерный классификатор
        # TfidfVectorizer => SVM => result
        pipe = Pipeline([
                ('vect', TfidfVectorizer(lowercase=False, analyzer='word', binary='True', token_pattern=r'\b\w+\b')),
                ('clf', svm.LinearSVC(C=5, verbose=True, max_iter=5000))
        ])
        # используем случайную перекрестную кросс-валидацию
        cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
        # набор параметров для выбора оптимального
        param_grid = [
            {
                'vect__ngram_range': [(1,2)],
                'vect__min_df': [1],
                'vect__smooth_idf': [True],
                'vect__sublinear_tf': [True],
                'vect__norm' : ['l2']

            }
        ]
        # поиск оптимальных параметров по сетке
        grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv)
        grid.fit(self.X_data, self.y_target)
        print(grid.best_params_)
        print(grid.best_score_)
        
        return grid.best_estimator_

In [11]:
# обучение классификатора на тренировочных данных
ml = MLearning(X_train.list_text, group_id_tr)
clf = ml.train_clf()
# предсказание классификатора на тестовых данных
Y = clf.predict(X_test.list_text)
# запись резульатов в файл
pd.DataFrame({'GROUP_ID' : Y,
              'id' : id_test})[['id', 'GROUP_ID']].to_csv(r'output\res.csv', index=False)

[LibLinear][LibLinear][LibLinear][LibLinear]{'vect__min_df': 1, 'vect__ngram_range': (1, 2), 'vect__norm': 'l2', 'vect__smooth_idf': True, 'vect__sublinear_tf': True}
0.935990770577
